### 📌 Feature Selection Based on XGBoost Feature Importance  
According to the **XGBoost Feature Importance** technique, `SW` and `Klogh` do not significantly impact the **Rate of Penetration (ROP)**. Therefore, we exclude these parameters and consider only the following features:  

- **`Depth`**: Depth of drilling  
- **`WOB`**: Weight on Bit  
- **`SURF_RPM`**: Rotation Per Minute (RPM)  
- **`PHIF`**: Porosity  
- **`VSH`**: Volume of Shale

In [29]:
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler

In [45]:
data = pd.read_csv("dataset_1.csv")

In [47]:
data.describe()

,Depth,WOB,SURF_RPM,ROP_AVG,PHIF,VSH,SW,KLOGH
count,151.000000,151.000000,151.000000,151.000000,151.000000,151.000000,151.000000,151.000000
mean,3697.417219,45393.934391,2.034981,0.007800,0.084423,0.299809,0.975579,37.072228
std,227.169433,15784.246882,0.208492,0.001476,0.068230,0.264596,0.108946,127.186210
min,3305.000000,16961.916000,1.314720,0.002666,0.002968,0.049451,0.351393,-0.001124
25%,3502.500000,34320.946500,1.998711,0.007368,0.037550,0.108539,1.000000,0.001000
50%,3700.000000,44243.480000,1.999697,0.008157,0.059274,0.193100,1.000000,0.001000
75%,3887.500000,53212.685000,2.001069,0.008760,0.097212,0.387664,1.000000,0.001000
max,4085.000000,97087.882000,2.639233,0.010447,0.279346,1.000000,1.013335,709.158935


In [49]:
#target = "ROP_AVG"
X = data[["Depth", "WOB", "SURF_RPM", "PHIF", "VSH"]]
y = data["ROP_AVG"]

In [51]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [53]:
# Normalize/Standardize data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [55]:
from xgboost import XGBRegressor

In [57]:
# Train an XGBoost surrogate model
surrogate_model = XGBRegressor(n_estimators=100, random_state=42)
surrogate_model.fit(X_train, y_train)

# Define the objective function for Bayesian Optimization
def objective_function(Depth, WOB, SURF_RPM, PHIF, VSH):
    # Scale the input
    X_new = scaler.transform([[Depth, WOB, SURF_RPM, PHIF, VSH]])
    # Predict ROP_AVG using the XGBoost model
    rop = surrogate_model.predict(X_new)
    return rop[0]  # Return the predicted ROP_AVG

In [61]:
pip install bayesian-optimization

Note: you may need to restart the kernel to use updated packages.


In [63]:
from bayes_opt import BayesianOptimization

In [65]:
# Define parameter bounds for Bayesian Optimization
pbounds = {
    "Depth": (data["Depth"].min(), data["Depth"].max()),  # Bounds for Depth
    "WOB": (data["WOB"].min(), data["WOB"].max()),        # Bounds for WOB
    "SURF_RPM": (data["SURF_RPM"].min(), data["SURF_RPM"].max()),  # Bounds for SURF_RPM
    "PHIF": (data["PHIF"].min(), data["PHIF"].max()),      # Bounds for PHIF
    "VSH": (data["VSH"].min(), data["VSH"].max())          # Bounds for VSH
}

In [67]:
# Initialize Bayesian Optimization
optimizer = BayesianOptimization(
    f=objective_function,
    pbounds=pbounds,
    random_state=42
)

In [69]:
 # Run optimization
optimizer.maximize(init_points=5, n_iter=25)  # Adjust init_points and n_iter as needed

# Get the best parameters
best_params = optimizer.max
print("Best Parameters:", best_params)

|   iter    |  target   |   Depth   |   PHIF    | SURF_RPM  |    VSH    |    WOB    |
-------------------------------------------------------------------------------------
| 1         | 0.008541  | 3.597e+03 | 0.2657    | 2.284     | 0.6185    | 2.946e+04 |
| 2         | 0.007122  | 3.427e+03 | 0.01902   | 2.462     | 0.6208    | 7.37e+04  |
| 3         | 0.006057  | 3.321e+03 | 0.271     | 2.417     | 0.2513    | 3.153e+04 |
| 4         | 0.007122  | 3.448e+03 | 0.08705   | 2.01      | 0.46      | 4.03e+04  |
| 5         | 0.004326  | 3.782e+03 | 0.04152   | 1.702     | 0.3977    | 5.35e+04  |


C:\Users\User\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\User\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\User\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\User\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\User\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\User\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature

| 6         | 0.004326  | 3.607e+03 | 0.194     | 1.657     | 0.8403    | 6.582e+04 |
| 7         | 0.004326  | 4.012e+03 | 0.03209   | 1.51      | 0.929     | 7.872e+04 |


C:\Users\User\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\User\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


| 8         | 0.004326  | 3.761e+03 | 0.01617   | 1.362     | 0.1488    | 3.565e+04 |
| 9         | 0.008541  | 3.586e+03 | 0.0168    | 2.534     | 0.9481    | 2.924e+04 |


...

C:\Users\User\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\User\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


| 26        | 0.008195  | 4.068e+03 | 0.2632    | 2.559     | 0.4813    | 4.697e+04 |
| 27        | 0.007122  | 3.402e+03 | 0.2604    | 2.336     | 0.5368    | 9.708e+04 |


C:\Users\User\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\User\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


| 28        | 0.008541  | 3.607e+03 | 0.08267   | 2.04      | 0.1528    | 2.923e+04 |
| 29        | 0.008605  | 3.812e+03 | 0.2383    | 1.813     | 0.964     | 3.375e+04 |
| 30        | 0.008605  | 3.994e+03 | 0.1874    | 1.836     | 0.2015    | 3.949e+04 |
Best Parameters: {'target': 0.008605283685028553, 'params': {'Depth': 3978.827820487244, 'PHIF': 0.1610870462271162, 'SURF_RPM': 1.8660667076689712, 'VSH': 0.2236417294718831, 'WOB': 39513.85444566769}}


C:\Users\User\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [71]:
# Evaluate the best parameters on the test set
best_rop = objective_function(**best_params["params"])
print("Optimized ROP_AVG:", best_rop)

Optimized ROP_AVG: 0.008605284


C:\Users\User\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [74]:
# Compare with baseline performance
baseline_rop = surrogate_model.predict(X_test).mean()
print("Baseline ROP_AVG:", baseline_rop)

Baseline ROP_AVG: 0.00779683


---
The **magnitude of improvement** on ROP_AVG is calculated as follows:

- **Improvement** = Optimized ROP_AVG - Baseline ROP_AVG  
  \( 0.008605284 - 0.00779683 = 0.000808454 \)

- **Percentage Improvement** = \( \frac{{\text{Improvement}}}{{\text{Baseline ROP_AVG}}} \times 100 \)  
  \( \frac{{0.000808454}}{{0.00779683}} \times 100 \approx 10.4\% \)

A **10.4% improvement** is considered significant, especially in industrial applications like drilling, where even small improvements can lead to **substantial cost savings** or **efficiency gains** over time. However, to ensure robustness, we will also compare the results of **Bayesian Optimization** with other **optimization techniques**

---